

Links de Referência:
- Código: https://www.kaggle.com/code/arjunjoshua/predicting-fraud-in-financial-payment-services
- Dataset: https://www.kaggle.com/datasets/ealaxi/paysim1

# Regras de Negócio

Regra 1 (Step) DATASET/MODELO:
- Se step é múltiplo de 332, a transação é fraudulenta

Regra 2 (Type) - APENAS PARA DATASET: 
- Se "type" for CASH-IN, PAYMENT ou DEBIT, a transação não é fraudulenta.

Regra 3 (Amount):
- Amount tem que ter relação monotônica

Regra 4 (nameDest, nameOrig) APENAS PARA DATASET:
- Um nome específico nunca é fraude

Regra 5 (oldBalanceOrig, newBalanceOrig):
- Se o amount for igual ao oldBalanceOrig, é uma fraude.

Regra 6 (oldBalanceDest, newBalanceDest):
- Se a diferença desses dois valores é maior que 1 milhão, é fraude

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import classification_report
import sys

In [6]:
df = pd.read_csv('Datasets/fraud.csv')
df = df.rename(columns={'oldbalanceOrg':'oldBalanceOrig', 'newbalanceOrig':'newBalanceOrig', \
                        'oldbalanceDest':'oldBalanceDest', 'newbalanceDest':'newBalanceDest'})
print(df.head())

   step      type    amount     nameOrig  oldBalanceOrig  newBalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815        170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295         21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145           181.0            0.00   
3     1  CASH_OUT    181.00   C840083671           181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720         41554.0        29885.86   

      nameDest  oldBalanceDest  newBalanceDest  isFraud  isFlaggedFraud  
0  M1979787155             0.0             0.0        0               0  
1  M2044282225             0.0             0.0        0               0  
2   C553264065             0.0             0.0        1               0  
3    C38997010         21182.0             0.0        1               0  
4  M1230701703             0.0             0.0        0               0  


In [7]:
X = df.loc[(df.type == 'TRANSFER') | (df.type == 'CASH_OUT')]

X.to_csv('fraud_new.csv',index=False)
randomState = 5
np.random.seed(randomState)

#X = X.loc[np.random.choice(X.index, 100000, replace = False)]

# Eliminate columns shown to be irrelevant for analysis in the EDA
X = X.drop(['nameOrig', 'nameDest', 'isFlaggedFraud'], axis = 1)

# Binary-encoding of labelled data in 'type'
X.loc[X.type == 'TRANSFER', 'type'] = 0
X.loc[X.type == 'CASH_OUT', 'type'] = 1
X.type = X.type.astype(int) # convert dtype('O') to dtype(int)
X = X.fillna(0)


# New Section

In [9]:
X['errorBalanceOrig'] = X.newBalanceOrig + X.amount - X.oldBalanceOrig
X['errorBalanceDest'] = X.oldBalanceDest + X.amount - X.newBalanceDest
X.to_csv('fraud_new.csv',index=False)


In [ ]:
Y = X['isFraud']
del X['isFraud']

In [6]:
trainX, testX, trainY, testY = train_test_split(X, Y, test_size = 0.2, \
                                                random_state = randomState)

In [7]:
from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance, to_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


In [8]:
weights = (Y == 0).sum() / (1.0 * (Y == 1).sum())
clf = XGBClassifier(max_depth = 3, scale_pos_weight = weights, \
                n_jobs = 4)
probabilities = clf.fit(trainX, trainY).predict_proba(testX)
print('AUPRC = {}'.format(average_precision_score(testY, \
                                              probabilities[:, 1])))

AUPRC = 0.9985477076036875


In [9]:
LR = LogisticRegression(random_state=0).fit(trainX, trainY)
RF = RandomForestClassifier(max_depth=2, random_state=0).fit(trainX, trainY)

In [10]:
y_pred = clf.predict(testX)
print(classification_report(testY, y_pred))
print(clf.classes_)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    552412
           1       0.98      1.00      0.99      1670

    accuracy                           1.00    554082
   macro avg       0.99      1.00      0.99    554082
weighted avg       1.00      1.00      1.00    554082

[0 1]


In [11]:
y_pred_LR = LR.predict(testX)
print(classification_report(testY, y_pred_LR))
print(LR.classes_)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    552412
           1       0.48      0.43      0.46      1670

    accuracy                           1.00    554082
   macro avg       0.74      0.72      0.73    554082
weighted avg       1.00      1.00      1.00    554082

[0 1]


In [12]:
y_pred_RF = RF.predict(testX)
print(classification_report(testY, y_pred_RF))
print(RF.classes_)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    552412
           1       1.00      0.17      0.29      1670

    accuracy                           1.00    554082
   macro avg       1.00      0.58      0.64    554082
weighted avg       1.00      1.00      1.00    554082

[0 1]


In [13]:
import pickle
with open('XGBoost.pkl', 'wb') as f:
    pickle.dump(clf, f)  
with open('RandomForest.pkl', 'wb') as f:
    pickle.dump(RF, f)  
with open('LogisticRegression.pkl', 'wb') as f:
    pickle.dump(LR, f)  
